In [2]:
#Installs
!pip install transformers
!pip install transformers[sentencepiece] sacrebleu
!pip install sentence-transformers

In [3]:
#Imports
import pandas as pd
import numpy as np
import tensorflow as tf
import csv
import nltk
import torch
import time
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from nltk.translate.bleu_score import corpus_bleu
from sentence_transformers import SentenceTransformer, util
from sacrebleu.metrics import CHRF
from nltk.translate.meteor_score import meteor_score
from nltk.lm import KneserNeyInterpolated
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.util import ngrams

In [4]:
pipe = pipeline("text2text-generation", model="facebook/m2m100_418M")
tokenizer = AutoTokenizer.from_pretrained("facebook/m2m100_418M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/m2m100_418M").to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [5]:
if torch.cuda.is_available():
    print("CUDA is available. Model will run on GPU.")
else:
    print("CUDA is not available. Model will run on CPU.")

CUDA is available. Model will run on GPU.


In [8]:
#Load Dataset
df=pd.read_csv('./Bhagwad_Gita.csv')
hindi_sentences=df['HinMeaning'].values
english_sentences=df['EngMeaning'].values

In [9]:
df.head()

,ID,Chapter,Verse,Shloka,Transliteration,HinMeaning,EngMeaning,WordMeaning
0,BG1.1,1,1,धृतराष्ट्र उवाच |\nधर्मक्षेत्रे कुरुक्षेत्रे स...,dhṛtarāṣṭra uvāca .\ndharmakṣetre kurukṣetre s...,।।1.1।।धृतराष्ट्र ने कहा -- हे संजय ! धर्मभूमि...,1.1 Dhritarashtra said What did my people and...,1.1 धर्मक्षेत्रे on the holy plain? कुरुक्षेत्...
1,BG1.2,1,2,सञ्जय उवाच |\nदृष्ट्वा तु पाण्डवानीकं व्यूढं द...,sañjaya uvāca .\ndṛṣṭvā tu pāṇḍavānīkaṃ vyūḍha...,।।1.2।।संजय ने कहा -- पाण्डव-सैन्य की व्यूह रच...,1.2. Sanjaya said Having seen the army of the...,1.2 दृष्ट्वा having seen? तु indeed? पाण्डवानी...
2,BG1.3,1,3,पश्यैतां पाण्डुपुत्राणामाचार्य महतीं चमूम् |\n...,paśyaitāṃ pāṇḍuputrāṇāmācārya mahatīṃ camūm .\...,।।1.3।।हे आचार्य ! आपके बुद्धिमान शिष्य द्रुपद...,"1.3. ""Behold, O Teacher! this mighty army of t...",1.3 पश्य behold? एताम् this? पाण्डुपुत्राणाम् ...
3,BG1.4,1,4,अत्र शूरा महेष्वासा भीमार्जुनसमा युधि |\nयुयुध...,atra śūrā maheṣvāsā bhīmārjunasamā yudhi .\nyu...,।।1.4।।इस सेना में महान् धनुर्धारी शूर योद्धा ...,"1.4. Here are heroes, mighty archers, eal in b...",1.4 अत्र here? शूराः heroes? महेष्वासाः mighty...
4,BG1.5,1,5,धृष्टकेतुश्चेकितानः काशिराजश्च वीर्यवान् |\nपु...,dhṛṣṭaketuścekitānaḥ kāśirājaśca vīryavān .\np...,"।।1.5।।धृष्टकेतु, चेकितान, बलवान काशिराज, पुर...","1.5. ""Dhrishtaketu, chekitana and the valiant ...",1.5 धृष्टकेतुः Dhrishtaketu? चेकितानः Chekitan...


In [10]:
Shlokas=df['Shloka'].values
hindi_sentences=df['HinMeaning'].values
english_sentences=df['EngMeaning'].values

In [11]:
start_time = time.time()
translatedEngHin = []

for index, (hindi, english) in enumerate(zip(hindi_sentences, english_sentences)):
    print(index+1)
    print("English:", english)
    encoded_en = tokenizer(english, return_tensors="pt", padding=True, truncation=True).to("cuda")
    generated_tokens = model.generate(**encoded_en, forced_bos_token_id=tokenizer.get_lang_id("hi"))
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    print("Translated: ", translation)
    print("Original: ", hindi)
    print("------------------------------")
    translatedEngHin.append(translation)

end_time = time.time()
total_time = end_time - start_time

print(f"Total time taken for translating first 100 lines: {total_time:.2f} seconds")


1
English: 1.1 Dhritarashtra said  What did my people and the sons of Pandu do when they had assembled
together eager for battle on the holy plain of Kurukshetra, O Sanjaya.
Translated:  1. Dhritarashtra ने कहा कि मेरे लोगों और पंडू के पुत्रों ने क्या किया जब वे एक साथ इकट्ठा हुए थे और कुरक्सा के पवित्र मंजिल पर लड़ाई के लिए उत्सुक थे, O Sanjaya।
Original:  ।।1.1।।धृतराष्ट्र ने कहा -- हे संजय ! धर्मभूमि कुरुक्षेत्र में एकत्र हुए युद्ध के इच्छुक (युयुत्सव:) मेरे और पाण्डु के पुत्रों ने क्या किया?
------------------------------
2
English: 1.2. Sanjaya said  Having seen the army of the Pandavas drawn up in battle-array,
King Duryodhana then approached his teacher (Drona) and spoke these words.
Translated:  सानियाया ने कहा कि पांडावस की सेना को लड़ाकू रस्सी में उठाकर देखा, राजा डुरीओडाना फिर अपने शिक्षक (ड्रोना) के पास आया और इन शब्दों को बोला।
Original:  ।।1.2।।संजय ने कहा -- पाण्डव-सैन्य की व्यूह रचना देखकर राजा दुर्योधन ने आचार्य द्रोण के पास जाकर ये वचन कहे।
---------------------------

In [12]:
#Saving the results
comaparision = pd.DataFrame({'Original': hindi_sentences, 'Translated': translatedEngHin})
comaparision.to_csv('/content/BG-M2M100-Translated-Eng_Hin.csv', index=False)

###Calculating the various accuracy scores
---
```
*   BLEU: 39.57
*   CHRF: 34.3010
*   STS: 0.6324
*   METEOR: 0.2948
*   Original Perplexity: 19.5507
*   Translated Perplexity: 19.0058
```

In [13]:
#BLEU Score
references = [[hindi] for hindi in hindi_sentences]  # Create reference list of lists
candidates = translatedEngHin  # Model translations
bleu_score = corpus_bleu(references, candidates)
print(f"BLEU score: {bleu_score:.4f}")

BLEU score: 0.3083


In [ ]:
# STS Score
# Load the pre-trained model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Encode the original Hindi sentences and the translated sentences
embeddings1 = model.encode(hindi_sentences, convert_to_tensor=True, device=model.device)
embeddings2 = model.encode(translatedEngHin, convert_to_tensor=True, device=model.device)

# Compute cosine similarities between the embeddings
cosine_scores = util.cos_sim(embeddings1, embeddings2)

# Calculate the average STS score
sts_score = cosine_scores.diag().mean().item()
print(f"STS score: {sts_score:.4f}")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

STS score: 0.6324


In [ ]:
# CHRF
# Initialize CHRF metric
chrf = CHRF(word_order=1)

# Calculate CHRF score
chrf_score = chrf.corpus_score(translatedEngHin, [[hindi] for hindi in hindi_sentences]).score
print(f"CHRF score: {chrf_score:.4f}")

CHRF score: 34.3010


In [ ]:
# METEOR Score
# Download required NLTK data (if not already downloaded)
nltk.download('punkt')  # Download the 'punkt' resource for sentence tokenization
nltk.download('wordnet')

# Tokenize the sentences if they're not already tokenized
hindi_sentences_tokenized = [nltk.word_tokenize(sent) for sent in hindi_sentences]
translatedEngHin_tokenized = [nltk.word_tokenize(sent) for sent in translatedEngHin]

# Calculate METEOR score
from nltk.translate.meteor_score import meteor_score # Import the meteor_score function
meteor_scores = [meteor_score([ref], hyp) for ref, hyp in zip(hindi_sentences_tokenized, translatedEngHin_tokenized)]
average_meteor_score = sum(meteor_scores) / len(meteor_scores)

print(f"METEOR score: {average_meteor_score:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


METEOR score: 0.2948


In [ ]:
# PERPLEXITY
def train_model(sentences, n=2):
    tokenized = [['<s>'] + list(map(str.lower, nltk.word_tokenize(sent))) + ['</s>'] for sent in sentences]
    train_data, padded_vocab = padded_everygram_pipeline(n, tokenized)
    model = KneserNeyInterpolated(n)
    model.fit(train_data, padded_vocab)
    return model

def calculate_perplexity(model, sentences, n=2):
    perplexities = []
    for sent in sentences:
        tokens = ['<s>'] + list(map(str.lower, nltk.word_tokenize(sent))) + ['</s>']
        test_data = list(ngrams(tokens, n))
        try:
            pp = model.perplexity(test_data)
            perplexities.append(pp)
        except ValueError:
            pass  # Skip sentences that cause errors
    return np.mean(perplexities) if perplexities else float('inf')

# Train models
original_model = train_model(hindi_sentences)
translation_model = train_model(translatedEngHin)

# Calculate perplexities
original_perplexity = calculate_perplexity(original_model, hindi_sentences)
translated_perplexity = calculate_perplexity(translation_model, translatedEngHin)

print(f"Original Perplexity: {original_perplexity:.4f}")
print(f"Translated Perplexity: {translated_perplexity:.4f}")

Original Perplexity: 19.5507
Translated Perplexity: 19.0058
